# I. Import data

In [1]:
import pandas as pd
import requests
import json
from pprint import pprint

In [2]:
path_1="Dataset\movies_metadata.csv"

In [3]:
data_1=pd.read_csv(path_1)

c:\users\corre\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_1=pd.DataFrame(data_1)

In [5]:
df_1.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
type(df_1.release_date[100])

str

### Clean up data formats to extract release year

In [7]:
df_1['Date'] =pd.to_datetime(df_1.release_date, errors = 'coerce')
df_1['Date'] = pd.to_datetime(df_1.Date, format = "%y/%m/%d")

In [8]:
df_1['Month'] = df_1.Date.dt.month
df_1['Year'] = df_1.Date.dt.year

In [9]:
df_1.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'Date', 'Month', 'Year'],
      dtype='object')

In [10]:
type(df_1.Year[0])

numpy.float64

### Generate the movies lists for api and scraping

In [146]:
# Approach 1
# imdb_id_list=[]
# for record in df_1.iterrows():
#     #print(record[1].Year)
#     if record[1].Year==2017:
#         print(record[1]['imdb_id'])
#         imdb_id_list.append(record[1].imdb_id)
# len(imdb_id_list)

532

In [16]:
# Approach 2
Year_2017=(df_1.Year==2017)
df_2017=df_1[Year_2017]
df_2017.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,Date,Month,Year
26560,False,"{'id': 295, 'name': 'Pirates of the Caribbean ...",230000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://pirates.disney.com/pirates-of-the-carib...,166426,tt1790809,en,Pirates of the Caribbean: Dead Men Tell No Tales,"Thrust into an all-new paycheck, a down-on-his...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Pirates of the Caribbean: Dead Men Tell No Tales,False,6.6,2814.0,2017-05-23,5.0,2017.0
26561,False,"{'id': 468551, 'name': 'Justice League Collect...",0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.justiceleaguethemovie.com,141052,tt0974015,en,Justice League,Fueled by his restored faith in humanity and i...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Post Production,NaN,Justice League,False,0.0,0.0,2017-11-15,11.0,2017.0
26565,False,"{'id': 131296, 'name': 'Thor Collection', 'pos...",0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://marvel.com/movies/movie/222/thor_ragnarok,284053,tt3501632,en,Thor: Ragnarok,Thor is imprisoned on the other side of the un...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Post Production,NaN,Thor: Ragnarok,False,0.0,0.0,2017-10-25,10.0,2017.0
26566,False,"{'id': 284433, 'name': 'Guardians of the Galax...",200000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://marvel.com/movies/movie/221/guardians_o...,283995,tt3896198,en,Guardians of the Galaxy Vol. 2,The Guardians must fight to keep their newfoun...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Obviously.,Guardians of the Galaxy Vol. 2,False,7.6,4858.0,2017-04-19,4.0,2017.0
30536,False,NaN,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",NaN,245842,tt2328678,en,The King's Daughter,King Louis XIV's quest for immortality leads h...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Post Production,What on Earth Can Hold More Power Than a King?,The King's Daughter,False,0.0,4.0,2017-12-30,12.0,2017.0


# II. IMDB API

In [26]:
# i for imdb id. 
url = "http://www.omdbapi.com/?i="
api_key = "&apikey=trilogy"

In [27]:
#for loop to incorporate the imdb id list from table.
response = requests.get(url + "" + api_key)
print(response.url)

http://www.omdbapi.com/?i=&apikey=trilogy


In [28]:
metascore_list=[]
for id in imdb_id_list:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        print(data['Metascore'])
        metascore_list.append(float(data['Metascore']))
    except:
        metascore_list.append('n/a')

39
45
74
67
N/A
N/A
56
N/A
47
76
N/A
43
40
72
N/A
N/A
N/A
58
40
52
48
N/A
75
49
38
70
68
46
45
N/A
72
73
N/A
N/A
33
75
95
78
43
72
69
37
34
47
N/A
42
37
71
75
N/A
50
N/A
N/A
50
56
N/A
25
N/A
N/A
N/A
28
N/A
75
84
77
62
55
67
55
58
86
71
75
41
65
N/A
50
N/A
N/A
76
93
50
N/A
66
84
52
N/A
N/A
64
N/A
33
49
N/A
N/A
45
52
46
46
27
84
N/A
N/A
N/A
N/A
N/A
N/A
N/A
28
22
N/A
N/A
43
70
N/A
73
38
54
64
N/A
77
55
N/A
N/A
N/A
61
85
86
N/A
63
N/A
74
N/A
N/A
N/A
74
N/A
N/A
N/A
N/A
74
N/A
56
N/A
56
N/A
72
N/A
76
26
76
61
52
N/A
N/A
N/A
32
N/A
54
N/A
N/A
62
N/A
N/A
63
N/A
N/A
44
65
N/A
63
74
49
N/A
N/A
57
N/A
N/A
25
N/A
N/A
N/A
N/A
57
71
N/A
74
N/A
N/A
79
N/A
40
28
40
50
56
N/A
N/A
37
N/A
65
N/A
41
34
N/A
N/A
37
N/A
40
54
53
N/A
57
N/A
15
60
N/A
N/A
N/A
44
N/A
N/A
50
N/A
22
52
39
75
N/A
N/A
37
45
45
34
N/A
56
N/A
63
N/A
40
N/A
72
65
53
77
69
N/A
67
78
66
63
N/A
N/A
59
65
N/A
N/A
41
N/A
N/A
N/A
31
61
70
57
N/A
N/A
33
N/A
N/A
N/A
56
60
57
N/A
N/A
46
65
54
66
68
68
N/A
68
45
N/A
89
N/A
N/A
46
42
60
N/A
N/A


In [29]:
len(metascore_list)

532

In [30]:
df_2017['imdb_metascore']=metascore_list

c:\users\corre\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_2017.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'Date', 'Month', 'Year',
       'imdb_metascore'],
      dtype='object')

In [32]:
# df_2017.to_csv('df_2017_metascore.csv')

In [33]:
Year_2016=(df_1.Year==2016)
df_2016=df_1[Year_2016]
imdb_id_2016=df_2016.imdb_id

metascore_list_2016=[]
for id in imdb_id_2016:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2016.append(float(data['Metascore']))
    except:
        metascore_list_2016.append('n/a')
        
df_2016=pd.DataFrame(imdb_id_2016)
df_2016['imdb_metascore']=metascore_list_2016
df_2016['title']=df_1.title[Year_2016]

In [34]:
df_2016.head()

,imdb_id,imdb_metascore,title
22325,tt2836166,n/a,Punk's Dead: SLC Punk 2
23634,tt2990756,75,Sound of Redemption: The Frank Morgan Story
25216,tt2429278,n/a,The Ghost and the Whale
26556,tt2638144,38,Ben-Hur
26557,tt0803096,32,Warcraft


In [35]:
Year_2015=(df_1.Year==2015)
df_2015=df_1[Year_2015]
imdb_id_2015=df_2015.imdb_id

metascore_list_2015=[]
for id in imdb_id_2015:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2015.append(float(data['Metascore']))
    except:
        metascore_list_2015.append('n/a')
        
df_2015=pd.DataFrame(imdb_id_2015)
df_2015['imdb_metascore']=metascore_list_2015
df_2015['title']=df_1.title[Year_2015]

In [36]:
Year_2014=(df_1.Year==2014)
df_2014=df_1[Year_2014]
imdb_id_2014=df_2014.imdb_id

metascore_list_2014=[]
for id in imdb_id_2014:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2014.append(float(data['Metascore']))
    except:
        metascore_list_2014.append('n/a')

df_2014=pd.DataFrame(imdb_id_2014)
df_2014['imdb_metascore']=metascore_list_2014
df_2014['title']=df_1.title[Year_2014]

In [37]:
Year_2013=(df_1.Year==2013)
df_2013=df_1[Year_2013]
imdb_id_2013=df_2013.imdb_id

metascore_list_2013=[]
for id in imdb_id_2013:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2013.append(float(data['Metascore']))
    except:
        metascore_list_2013.append('n/a')

df_2013=pd.DataFrame(imdb_id_2013)
df_2013['imdb_metascore']=metascore_list_2013
df_2013['title']=df_1.title[Year_2013]

In [38]:
Year_2012=(df_1.Year==2012)
df_2012=df_1[Year_2012]
imdb_id_2012=df_2012.imdb_id

metascore_list_2012=[]
for id in imdb_id_2012:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2012.append(float(data['Metascore']))
    except:
        metascore_list_2012.append('n/a')

df_2012=pd.DataFrame(imdb_id_2012)
df_2012['imdb_metascore']=metascore_list_2012
df_2012['title']=df_1.title[Year_2012]

In [39]:
Year_2011=(df_1.Year==2011)
df_2011=df_1[Year_2011]
imdb_id_2011=df_2011.imdb_id

metascore_list_2011=[]
for id in imdb_id_2011:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2011.append(float(data['Metascore']))
    except:
        metascore_list_2011.append('n/a')

df_2011=pd.DataFrame(imdb_id_2011)
df_2011['imdb_metascore']=metascore_list_2011
df_2011['title']=df_1.title[Year_2011]

In [40]:
Year_2010=(df_1.Year==2010)
df_2010=df_1[Year_2010]
imdb_id_2010=df_2010.imdb_id

metascore_list_2010=[]
for id in imdb_id_2010:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2010.append(float(data['Metascore']))
    except:
        metascore_list_2010.append('n/a')

df_2010=pd.DataFrame(imdb_id_2010)
df_2010['imdb_metascore']=metascore_list_2010
df_2010['title']=df_1.title[Year_2010]

In [41]:
Year_2009=(df_1.Year==2009)
df_2009=df_1[Year_2009]
imdb_id_2009=df_2009.imdb_id

metascore_list_2009=[]
for id in imdb_id_2009:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2009.append(float(data['Metascore']))
    except:
        metascore_list_2009.append('n/a')

df_2009=pd.DataFrame(imdb_id_2009)
df_2009['imdb_metascore']=metascore_list_2009
df_2009['title']=df_1.title[Year_2009]

In [42]:
Year_2008=(df_1.Year==2008)
df_2008=df_1[Year_2008]
imdb_id_2008=df_2008.imdb_id

metascore_list_2008=[]
for id in imdb_id_2008:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2008.append(float(data['Metascore']))
    except:
        metascore_list_2008.append('n/a')

df_2008=pd.DataFrame(imdb_id_2008)
df_2008['imdb_metascore']=metascore_list_2008
df_2008['title']=df_1.title[Year_2008]

In [43]:
Year_2007=(df_1.Year==2007)
df_2007=df_1[Year_2007]
imdb_id_2007=df_2007.imdb_id

metascore_list_2007=[]
for id in imdb_id_2007:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2007.append(float(data['Metascore']))
    except:
        metascore_list_2007.append('n/a')

df_2007=pd.DataFrame(imdb_id_2007)
df_2007['imdb_metascore']=metascore_list_2007
df_2007['title']=df_1.title[Year_2007]

In [44]:
Year_2006=(df_1.Year==2006)
df_2006=df_1[Year_2006]
imdb_id_2006=df_2006.imdb_id

metascore_list_2006=[]
for id in imdb_id_2006:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2006.append(float(data['Metascore']))
    except:
        metascore_list_2006.append('n/a')

df_2006=pd.DataFrame(imdb_id_2006)
df_2006['imdb_metascore']=metascore_list_2006
df_2006['title']=df_1.title[Year_2006]

In [45]:
Year_2005=(df_1.Year==2005)
df_2005=df_1[Year_2005]
imdb_id_2005=df_2005.imdb_id

metascore_list_2005=[]
for id in imdb_id_2005:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2005.append(float(data['Metascore']))
    except:
        metascore_list_2005.append('n/a')

df_2005=pd.DataFrame(imdb_id_2005)
df_2005['imdb_metascore']=metascore_list_2005
df_2005['title']=df_1.title[Year_2005]

In [46]:
Year_2004=(df_1.Year==2004)
df_2004=df_1[Year_2004]
imdb_id_2004=df_2004.imdb_id

metascore_list_2004=[]
for id in imdb_id_2004:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2004.append(float(data['Metascore']))
    except:
        metascore_list_2004.append('n/a')

df_2004=pd.DataFrame(imdb_id_2004)
df_2004['imdb_metascore']=metascore_list_2004
df_2004['title']=df_1.title[Year_2004]

In [47]:
Year_2003=(df_1.Year==2003)
df_2003=df_1[Year_2003]
imdb_id_2003=df_2003.imdb_id

metascore_list_2003=[]
for id in imdb_id_2003:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2003.append(float(data['Metascore']))
    except:
        metascore_list_2003.append('n/a')

df_2003=pd.DataFrame(imdb_id_2003)
df_2003['imdb_metascore']=metascore_list_2003
df_2003['title']=df_1.title[Year_2003]

In [48]:
Year_2002=(df_1.Year==2002)
df_2002=df_1[Year_2002]
imdb_id_2002=df_2002.imdb_id

metascore_list_2002=[]
for id in imdb_id_2002:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2002.append(float(data['Metascore']))
    except:
        metascore_list_2002.append('n/a')

df_2002=pd.DataFrame(imdb_id_2002)
df_2002['imdb_metascore']=metascore_list_2002
df_2002['title']=df_1.title[Year_2002]

In [49]:
Year_2001=(df_1.Year==2001)
df_2001=df_1[Year_2001]
imdb_id_2001=df_2001.imdb_id

metascore_list_2001=[]
for id in imdb_id_2001:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2001.append(float(data['Metascore']))
    except:
        metascore_list_2001.append('n/a')

df_2001=pd.DataFrame(imdb_id_2001)
df_2001['imdb_metascore']=metascore_list_2001
df_2001['title']=df_1.title[Year_2001]

In [50]:
Year_2000=(df_1.Year==2000)
df_2000=df_1[Year_2000]
imdb_id_2000=df_2000.imdb_id

metascore_list_2000=[]
for id in imdb_id_2000:
    try:
        response = requests.get(url + id + api_key)
        data=response.json()
        metascore_list_2000.append(float(data['Metascore']))
    except:
        metascore_list_2000.append('n/a')

df_2000=pd.DataFrame(imdb_id_2000)
df_2000['imdb_metascore']=metascore_list_2000
df_2000['title']=df_1.title[Year_2000]

In [51]:
df_2017=df_2017[['imdb_id','imdb_metascore','title']]
df_2017.head()

,imdb_id,imdb_metascore,title
26560,tt1790809,39,Pirates of the Caribbean: Dead Men Tell No Tales
26561,tt0974015,45,Justice League
26565,tt3501632,74,Thor: Ragnarok
26566,tt3896198,67,Guardians of the Galaxy Vol. 2
30536,tt2328678,n/a,The King's Daughter


In [52]:
df_2017.to_csv('df_2017_imdb.csv')
df_2016.to_csv('df_2016_imdb.csv')
df_2015.to_csv('df_2015_imdb.csv')
df_2014.to_csv('df_2014_imdb.csv')
df_2013.to_csv('df_2013_imdb.csv')
df_2012.to_csv('df_2012_imdb.csv')
df_2011.to_csv('df_2011_imdb.csv')
df_2010.to_csv('df_2010_imdb.csv')
df_2009.to_csv('df_2009_imdb.csv')
df_2008.to_csv('df_2008_imdb.csv')
df_2007.to_csv('df_2007_imdb.csv')
df_2006.to_csv('df_2006_imdb.csv')
df_2005.to_csv('df_2005_imdb.csv')
df_2004.to_csv('df_2004_imdb.csv')
df_2003.to_csv('df_2003_imdb.csv')
df_2002.to_csv('df_2002_imdb.csv')
df_2001.to_csv('df_2001_imdb.csv')
df_2000.to_csv('df_2000_imdb.csv')

# III. Scraping Rotten Tomato Ratings with the movie lists

In [53]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time

In [120]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.rottentomatoes.com/'
browser.visit(url)

In [76]:
new_2017=pd.read_csv('second_elt.csv')
new_2017.head()

,Unnamed: 0,Title,Distributor,Gross,Max Th,Opening,% of Total,Open Th,Open,Close,...,BoxOffice,Director,Writer,Actors,Runtime,Plot,IMDB_ID,IMDB_Rating,IMDB_Votes,IMDB_Metascore
0,0,Star Wars: Episode VIII - The Last Jedi,Walt Disney Studios Motion Pictures,"$620,181,382",4232,"$220,009,584",35.5%,4232,Dec 15,Apr 19,...,"$619,117,636",Rian Johnson,"Rian Johnson, George Lucas (based on character...","Mark Hamill, Carrie Fisher, Adam Driver, Daisy...",152 min,Rey develops her newly discovered abilities wi...,tt2527336,7.1,"481,503",85.0
1,1,Beauty and the Beast,Walt Disney Studios Motion Pictures,"$504,014,165",4210,"$174,750,616",34.7%,4210,Mar 17,Jul 13,...,"$47,611,331","Gary Trousdale, Kirk Wise","Linda Woolverton (animation screenplay by), Br...","Robby Benson, Jesse Corti, Rex Everhart, Angel...",84 min,A prince cursed to spend his days as a hideous...,tt0101414,8.0,"396,910",95.0
2,2,Guardians of the Galaxy Vol. 2,Walt Disney Studios Motion Pictures,"$389,813,101",4347,"$146,510,104",37.6%,4347,May 5,Sep 21,...,"$389,804,217",James Gunn,"James Gunn, Dan Abnett (based on the Marvel co...","Chris Pratt, Zoe Saldana, Dave Bautista, Vin D...",136 min,The Guardians struggle to keep together as a t...,tt3896198,7.6,"507,090",67.0
3,3,It,Warner Bros.,"$327,481,748",4148,"$123,403,419",37.7%,4103,Sep 8,Dec 14,...,"$326,898,358",Andy Muschietti,"Chase Palmer (screenplay by), Cary Joji Fukuna...","Jaeden Martell, Jeremy Ray Taylor, Sophia Lill...",135 min,"In the summer of 1989, a group of bullied kids...",tt1396484,7.3,"404,288",69.0
4,4,Thor: Ragnarok,Walt Disney Studios Motion Pictures,"$315,058,289",4080,"$122,744,989",39%,4080,Nov 3,Mar 15,...,"$314,971,245",Taika Waititi,"Eric Pearson, Craig Kyle, Christopher L. Yost,...","Chris Hemsworth, Tom Hiddleston, Cate Blanchet...",130 min,"Imprisoned on the planet Sakaar, Thor must rac...",tt3501632,7.9,"516,330",74.0


In [77]:
# Chopping up the full list into 3 groups.
movies_2017_1=new_2017.Title[:221]
movies_2017_2=new_2017.Title[221:451]
movies_2017_3=new_2017.Title[451:]
movies_2017_1

0      Star Wars: Episode VIII - The Last Jedi
1                         Beauty and the Beast
2               Guardians of the Galaxy Vol. 2
3                                           It
4                               Thor: Ragnarok
                        ...                   
216                            Railroad Tigers
217                               City of Rock
218               The Killing of a Sacred Deer
219                               Kung Fu Yoga
220                         The Lost City of Z
Name: Title, Length: 221, dtype: object

In [81]:
movies_rot_2017_1=[]

for movie in movies_2017_1:

    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0] 
    search_button.click()
    
    time.sleep(8)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2017' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2017_1.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(movie,result.find('span',class_="search__results-item-score").text)
        except:
            pass

Beauty and the Beast 
                                71%
                            
Guardians of the Galaxy Vol. 2 
                                84%
                            
Thor: Ragnarok 
                                93%
                            
Spider-Man: Homecoming 
                                92%
                            
Wonder Woman 
                                93%
                            
The Fate of the Furious 
                                67%
                            
Justice League 
                                40%
                            
Logan 
                                93%
                            
Despicable Me 3 
                                58%
                            
Pirates of the Caribbean: Dead Men Tell No Tales 
                                30%
                            
Kong: Skull Island 
                                75%
                            
War for the Planet of the Apes 
          

Stronger 
                                90%
                            
Table 19 
                                26%
                            
All Saints 
                                95%
                            
Collide 
                                21%
                            
The Book of Henry 
                                22%
                            
The Man Who Invented Christmas 
                                79%
                            
Jab Harry Met Sejal 
                                10%
                            
The Disaster Artist 
                                91%
                            
The Resurrection of Gavin Stone 
                                50%
                            
Tulip Fever 
                                10%
                            
LBJ 
                                57%
                            
A Question of Faith 
                                40%
                            
Golmaal Again 

In [83]:
movies_rot_2017_2=[]

for movie in movies_2017_2:

    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0] 
    search_button.click()
    
    time.sleep(8)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2017' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2017_2.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(movie,result.find('span',class_="search__results-item-score").text)
        except:
            pass

3 Generations 
                                32%
                            
A Ghost Story 
                                91%
                            
Brad's Status 
                                79%
                            
The Bad Batch 
                                47%
                            
The Swindlers 
                                40%
                            
Personal Shopper 
                                81%
                            
Risk 
                                81%
                            
The Square 
                                85%
                            
Patti Cake$ 
                                83%
                            
A United Kingdom 
                                84%
                            
Wonderstruck 
                                68%
                            
Munna Michael 
                                50%
                            
Lady Macbeth 
                                88%

Buster's Mal Heart 
                                72%
                            
Bobbi Jene 
                                59%
                            
Bill Nye: Science Guy 
                                100%
                            
The Farthest 
                                100%
                            
The Last Shaman 
                                40%
                            
Naples '44 
                                63%
                            
The Persian Connection 
                                63%
                            
Quest 
                                98%
                            
All the Queen's Horses 
                                80%
                            
Mansfield 66/67 
                                79%
                            
Vince Giordano: There's a Future in the Past 
                                100%
                            
XX 
                                70%
                          

In [85]:
movies_rot_2017_3=[]
counter=0

for movie in movies_2017_3:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0] 
    search_button.click()
    
    time.sleep(8)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2017' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2017_3.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

1
2
4
6
11
15
19
22
23
24
27
28
29
33
34
36
39
43
44
50
52
57
58
64
67
69
72
73
75
76
79
80
81
83
84
85
86
87
88
89
90
95
97
98
101
102
104
105
106
107
108
110
116
117
119
121
124
126
132
135
141
142
143
144
148
149
152
153
157
163
167
169
175
177
178
180
186
188
190
193
194
201
204
205
214
219
223


In [97]:
# Concatenate the 3 list together into one list.
movies_rot_2017=movies_rot_2017_1+movies_rot_2017_2+movies_rot_2017_3
len(movies_rot_2017)

360

In [127]:
# Turn data into a dataframe.
df_2=pd.DataFrame(movies_rot_2017)
df_2.columns=['title','rotten_tomato_meter']
df_cleaned=df_2.replace('\n','', regex=True)
rmmeter_cleaned=df_cleaned.rotten_tomato_meter.replace(' ','',regex=True)
df_clean_full = df_cleaned
df_clean_full.rotten_tomato_meter=rmmeter_cleaned
df_clean_full.rotten_tomato_meter.describe()

count      360
unique      91
top       100%
freq        27
Name: rotten_tomato_meter, dtype: object

In [135]:
# Convert the rating data from string to float.
def p2f(x):
    return(float(x.strip('%'))/100)

rmmeter_float=[]

for number in rmmeter_cleaned:
    rmmeter_float.append(p2f(number))


In [141]:
# Calculate the mean of the data to replace 'nan' in the full dataset.
df_clean_full.rotten_tomato_meter=rmmeter_float
df_clean_full.rotten_tomato_meter.mean()

0.6714444444444444

In [143]:
df_clean_full.to_csv('rotten_tomato_floated_2017.csv')

### Note: Tried to proceed with the rest of the years from 2000. But eventually had to abort due to time limit. 

In [56]:
movies_2016=list(df_2016.title)
len(movies_2016)

1604

In [89]:
# seperate the list into groups of 200-ish each.
movies_2016_1=movies_2016[:201]
movies_2016_2=movies_2016[201:401]
movies_2016_3=movies_2016[401:601]
movies_2016_4=movies_2016[601:801]
movies_2016_5=movies_2016[801:1001]
movies_2016_6=movies_2016[1001:1201]
movies_2016_7=movies_2016[1201:1401]
movies_2016_8=movies_2016[1401:]

In [92]:
movies_rot_2016_1=[]
counter=0

for movie in movies_2016_1:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0]
    search_button.click()
    
    time.sleep(8)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2016' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2016_1.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

4
5
6
7
8
9
11
12
13
16
17
18
19
21
22
25
26
28
30
35
36
37
42
43
48
52
53
54
56
57
58
59
60
61
62
63
64
70
73
74
75
76
78
81
82
83
89
90
91
92
93
95
96
98
100
107
108
109
110
112
113
115
116
118
119
120
121
125
128
135
139
147
149
152
153
154
159
160
161
162
165
167
168
171
176
178
181
185
187
188
189
190
191
193
194
196
199
200
201


In [94]:
movies_rot_2016_2=[]
counter=0

for movie in movies_2016_2:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0]
    search_button.click()
    
    time.sleep(8)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2016' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2016_2.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

1
3
4
5
7
9
11
12
13
25
26
27
28
30
31
34
36
40
42
43
44
45
49
51
56
57
58
60
61
62
63
64
71
73
74
77
78
81
83
100
101
103
104
108
109
111
112
114
115
117
120
121
122
135
137
138
145
146
148
150
153
160
161
163
164
165
167
171
177
178
178
179
180
181
183
184
185
189
190
194
196
197
199
200


In [101]:
movies_rot_2016_3=[]
counter=0

for movie in movies_2016_3:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0]
    search_button.click()
    
    time.sleep(8)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2016' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2016_3.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

1
3
4
6
7
8
10
11
12
13
15
16
17
20
21
23
27
30
32
37
40
46
49
50
55
56
57
61
65
69
72
73
76
77
79
80
84
85
87
89
92
99
103
106
107
108
109
110
111
119
122
123
132
133
135
136
137
144
147
150
152
153
155
158
160
162
163
165
166
167
168
169
170
186
187
191
192
194
198
199
200


In [109]:
movies_rot_2016_1sthalf=movies_rot_2016_1+movies_rot_2016_2+movies_rot_2016_3
len(movies_rot_2016_1sthalf)
df_2016_1sthalf=pd.DataFrame(movies_rot_2016_1sthalf)
df_2016_1sthalf.to_csv('df_2016_1sthalf_rmmeter.csv')

In [121]:
movies_rot_2016_4=[]
counter=0

for movie in movies_2016_4:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0]
    search_button.click()
    
    time.sleep(10)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2016' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2016_4.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

1
4
105
109
111
116


ElementDoesNotExist: no elements could be found with xpath "//*[@id="fullscreen-search-term"]"

In [119]:
movies_rot_2016_5=[]
counter=0

for movie in movies_2016_5:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0]
    search_button.click()
    
    time.sleep(10)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2016' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2016_5.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

KeyboardInterrupt: 

In [ ]:
movies_rot_2016_6=[]
counter=0

for movie in movies_2016_6:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0]
    search_button.click()
    
    time.sleep(10)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2016' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2016_6.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

In [ ]:
movies_rot_2016_7=[]
counter=0

for movie in movies_2016_7:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0]
    search_button.click()
    
    time.sleep(10)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2016' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2016_7.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

In [ ]:
movies_rot_2016_8=[]
counter=0

for movie in movies_2016_8:
    counter+=1
    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0]
    search_button.click()
    
    time.sleep(10)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2016' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2016_8.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(counter)
        except:
            pass

In [95]:
len(movies_rot_2016_1), len(movies_rot_2016_2),len(movies_rot_2016_3),len(movies_rot_2016_4),\
len(movies_rot_2016_5),len(movies_rot_2016_6),len(movies_rot_2016_7),len(movies_rot_2016_8)

(99, 84)

In [ ]:
movies_rot_2016=movies_rot_2016_1+movies_rot_2016_2+movies_rot_2016_3+movies_rot_2016_4+movies_rot_2016_5+movies_rot_2016_6+\
movies_rot_2016_7+movies_rot_2016_8

In [85]:
df_rot_2016=pd.DataFrame(movies_rot_2016)
df_rot_2016.columns=['title','rotten_tomato_meter']
df_rot_2016_cleaned=df_rot_2016.replace('\n','', regex=True)
rmmeter_cleaned_2016=df_rot_2016_cleaned.rotten_tomato_meter.replace(' ','',regex=True)
df_rot_full_2016 = df_rot_2016_cleaned
df_rot_full_2016.rotten_tomato_meter=rmmeter_cleaned

In [86]:
movies_2015=list(df_2015.title)
len(movies_2015)

1905

In [ ]:
movies_rot_2015=[]

for movie in movies_2015:

    search_bar_xpath ='//*[@id="fullscreen-search-term"]'
    search_bar = browser.find_by_xpath(search_bar_xpath)[0]

    search_bar.clear()
    search_bar.fill(movie)

    search_button_xpath = '//*[@id="fullscreen-search-desktop-search-btn"]' 
    search_button = browser.find_by_xpath(search_button_xpath)[0] 
    search_button.click()
    
    time.sleep(5)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    results=soup.find_all('li',class_="search__results-item search__results-item--center")

    for result in results:
        try:
            if '2015' in str(result):
                if result.find('a',class_="search__results-item-info-name rt-font-frank-demi rt-font-sm").text==movie:
                    movies_rot_2015.append((movie,result.find('span',class_="search__results-item-score").text))
                    print(movie,result.find('span',class_="search__results-item-score").text)
        except:
            pass